In [202]:
import pandas as pd
import numpy as np

import tensorflow as tf
import sklearn
from sklearn.preprocessing import LabelEncoder
import math

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [203]:
data = pd.read_csv("iris.csv")
data.head(10)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa
6,4.6,3.4,1.4,0.3,setosa
7,5.0,3.4,1.5,0.2,setosa
8,4.4,2.9,1.4,0.2,setosa
9,4.9,3.1,1.5,0.1,setosa


In [204]:
le = LabelEncoder()
data['length'] = le.fit_transform(data['sepal_length'])
data['width'] = le.fit_transform(data['sepal_width'])
data['p_length'] = le.fit_transform(data['petal_length'])

data['p_width'] = le.fit_transform(data['petal_width'])

data["le_species"] = le.fit_transform(data["species"])

data.head()


,sepal_length,sepal_width,petal_length,petal_width,species,length,width,p_length,p_width,le_species
0,5.1,3.5,1.4,0.2,setosa,8,14,4,1,0
1,4.9,3.0,1.4,0.2,setosa,6,9,4,1,0
2,4.7,3.2,1.3,0.2,setosa,4,11,3,1,0
3,4.6,3.1,1.5,0.2,setosa,3,10,5,1,0
4,5.0,3.6,1.4,0.2,setosa,7,15,4,1,0


In [205]:

classes = max(data["le_species"]) + 1
classes

3

In [206]:

hidden_layer_size = 10



#data=data.iloc[np.random.permutation(len(data))] #shufflning data
data= shuffle(data)
data=data.reset_index(drop=True) #arranging index after shuffle
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species,length,width,p_length,p_width,le_species
0,5.2,3.5,1.5,0.2,setosa,9,14,5,1,0
1,5.9,3.0,4.2,1.5,versicolor,16,9,18,11,1
2,6.2,2.9,4.3,1.3,versicolor,19,8,19,9,1
3,4.8,3.0,1.4,0.1,setosa,5,9,4,0,0
4,6.6,2.9,4.6,1.3,versicolor,23,8,22,9,1


In [207]:
def read_dataset():
	df=pd.read_csv('iris.csv')
	X=df[df.columns[0:4]].values #features
	y1=df[df.columns[4]]  #label
	
	#encode tha label
	LE= LabelEncoder()
	
	y=LE.fit_transform(y1)
	Y = one_hot_encode(y)  
	
	return X,Y, y1



def one_hot_encode(species):
    batch_size = len(species)
    all_zero = np.zeros([batch_size, 3])
    for i in range(batch_size):
        all_zero[i][species[i]] = 1
    return all_zero


X, Y, y1 = read_dataset()


In [208]:
#creating test and train data

x_input, test_x , y_input, test_y = train_test_split(X, Y, test_size=0.20 , random_state=400)
x_input.shape


(120, 4)

In [209]:
n_nodes_hl = 10
x=tf.placeholder(tf.float32,shape=[None,4])
y=tf.placeholder(tf.float32, shape=[None, 3] )
n_classes = 3


In [210]:

weights = tf.Variable(tf.truncated_normal([4,n_nodes_hl], stddev = math.sqrt(1.0/n_nodes_hl)))
biases = tf.Variable(tf.random_normal([n_nodes_hl]))

l1 = tf.add(tf.matmul(x, weights), biases)
l1 = tf.nn.relu(l1)  #passing it through activation funct




In [211]:

output_layer = {'weights' : tf.Variable(tf.truncated_normal([n_nodes_hl, n_classes],stddev=0.1)) , 'biases' : tf.Variable(tf.zeros([3]))}




In [212]:

output = tf.matmul(l1, output_layer['weights']) + output_layer['biases']
output

<tf.Tensor 'add_7:0' shape=(?, 3) dtype=float32>

In [213]:
y_ = tf.nn.softmax(output)  #for multi class claasification
y_

<tf.Tensor 'Softmax_4:0' shape=(?, 3) dtype=float32>

In [214]:
#session parameters
learning_rate = 0.07
training_epochs = 10
cost_history = np.empty(shape=[1], dtype=float)
loss= tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_), reduction_indices=[1]))
optimize = tf.train.AdamOptimizer(learning_rate).minimize(loss)

init=tf.global_variables_initializer()

In [215]:
#loss= tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_), reduction_indices=[1]))
#loss = tf.losses.mean_squared_error(y_, y)
mse_history = []
accuracy_history =[]

In [216]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(15):
            o,loss1 = sess.run([optimize,loss], feed_dict={x: x_input, y: y_input})
            
            correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(y,1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
            
            accuracy_ = sess.run(accuracy, feed_dict={x: x_input, y: y_input})
            accuracy_history.append(accuracy_)
            print('epoch : ', epoch ,'  loss : ', loss1 , '   Train accuracy : ', accuracy_)
            
  

epoch :  0   loss :  1.3455077    Train accuracy :  0.33333334
epoch :  1   loss :  1.1297917    Train accuracy :  0.33333334
epoch :  2   loss :  1.0415882    Train accuracy :  0.34166667
epoch :  3   loss :  0.99517214    Train accuracy :  0.49166667
epoch :  4   loss :  0.9485775    Train accuracy :  0.6666667
epoch :  5   loss :  0.8706285    Train accuracy :  0.6666667
epoch :  6   loss :  0.77797496    Train accuracy :  0.6666667
epoch :  7   loss :  0.69373065    Train accuracy :  0.65833336
epoch :  8   loss :  0.6282327    Train accuracy :  0.65833336
epoch :  9   loss :  0.58478826    Train accuracy :  0.65833336
epoch :  10   loss :  0.55352783    Train accuracy :  0.65833336
epoch :  11   loss :  0.5183791    Train accuracy :  0.65833336
epoch :  12   loss :  0.4828866    Train accuracy :  0.85
epoch :  13   loss :  0.46036428    Train accuracy :  0.85833335
epoch :  14   loss :  0.44751662    Train accuracy :  0.925


In [108]:
y_


<tf.Tensor 'Softmax_1:0' shape=(?, 3) dtype=float32>

In [79]:
out= sess.run(y_, feed_dict={x: x_e})
out

array([[9.97436106e-01, 9.75685718e-04, 1.58818159e-03],
       [9.83584285e-01, 7.55506847e-03, 8.86061508e-03],
       [9.90133286e-01, 3.92754795e-03, 5.93919354e-03],
       [9.90090728e-01, 4.31533251e-03, 5.59398066e-03],
       [9.98193681e-01, 6.47496607e-04, 1.15890033e-03],
       [9.99689221e-01, 1.15903604e-04, 1.94853521e-04],
       [9.96024370e-01, 1.45231350e-03, 2.52326252e-03],
       [9.96860862e-01, 1.29291206e-03, 1.84618996e-03],
       [9.75110769e-01, 1.07803447e-02, 1.41089242e-02],
       [9.90411818e-01, 4.40751156e-03, 5.18062664e-03]], dtype=float32)

In [80]:
accuracy_


0.0

In [81]:
mse_


0.3727853522569214